## Step 1: Clone the PhysFormer repository and prepare the environment

In [ ]:

!git clone 'https://github.com/ZitongYu/PhysFormer.git'
!wget 'https://drive.usercontent.google.com/download?id=1n1TpMQfU-OkZdJglEJyFp-vGo9JXbgsT&export=download&authuser=0&confirm=t&uuid=6e3d51ce-76c0-43cc-8799-db0a79976c3a&at=APZUnTW_BrrV1iI05BG41_sccbpk%3A1710947798275'
!mv '/content/download?id=1n1TpMQfU-OkZdJglEJyFp-vGo9JXbgsT&export=download&authuser=0&confirm=t&uuid=6e3d51ce-76c0-43cc-8799-db0a79976c3a&at=APZUnTW_BrrV1iI05BG41_sccbpk:1710947798275' 'v.zip'
!unzip '/content/v.zip'
!wget 'https://drive.usercontent.google.com/download?id=1jBSbM88fA-beaoVi8ILFyL0SvVVMA9c9&export=download&authuser=0&confirm=t&uuid=d1551966-5490-4665-a90e-eb60816659ac&at=APZUnTUt_KWH-tRaMFc_eaBw1Hdq%3A1710948205447'
!mv '/content/download?id=1jBSbM88fA-beaoVi8ILFyL0SvVVMA9c9&export=download&authuser=0&confirm=t&uuid=d1551966-5490-4665-a90e-eb60816659ac&at=APZUnTUt_KWH-tRaMFc_eaBw1Hdq:1710948205447' 'Physformer_VIPL_fold1.pkl'
!mv '/content/Physformer_VIPL_fold1.pkl' '/content/PhysFormer'
%cd '/content/PhysFormer'


## Step 2: Load the required libraries and the pretrained model

In [ ]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from model import ViT_ST_ST_Compact3_TDC_gra_sharp




###  Initialize the model

In [ ]:
pytorch_model = ViT_ST_ST_Compact3_TDC_gra_sharp(image_size=(160, 128, 128), patches=(4, 4, 4), dim=96, ff_dim=144, num_heads=4, num_layers=12, dropout_rate=0.1, theta=0.7)
pytorch_model = pytorch_model.cuda()


###  Load pretrained weights

In [ ]:

pytorch_model.load_state_dict(torch.load('Physformer_VIPL_fold1.pkl'))  
pytorch_model.eval()

# Step 3: Prepare a sample video for inference

In [ ]:
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (128, 160))  
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Example usage with a sample video file
sample_video = preprocess_video('path_to_sample_video.mp4')  # Provide the path to your sample video
sample_video_tensor = torch.from_numpy(sample_video).float().cuda()

# Run inference on the sample video
with torch.no_grad():
    output = pytorch_model(sample_video_tensor)